In [1]:
import os
import struct

tot_items = 1000
topk = 10

In [2]:
vec_file = "../siftsmall/siftsmall_base.fvecs"
items = []

with open(vec_file, "rb") as f:
    dim = struct.unpack('i', f.read(4))[0]
    print(dim)
    f.seek(0) # seek back to 0, every row has a header containing the dim

    f_size = os.path.getsize("../siftsmall/siftsmall_base.fvecs")
    vec_cnt = f_size // ((dim + 1) * 4)

    for v in range(vec_cnt):
        f.read(4) # row header
        # for some reason unpack is misbehaving, need to unpack one by one...
        items.append([struct.unpack('f', f.read(4))[0] for _ in range(dim)])

128


In [3]:
from hnsw import HNSWIndex
from vector_compression import PQCompression

hnsw_ = HNSWIndex(M=32, ef_construction=100)

pq = PQCompression(M=32, K=256, D=128)
pq.Train(items)
hnsw_pq = HNSWIndex(M=32, ef_construction=100, compression=pq)

for v in range(tot_items):
    hnsw_.Insert(items[v])
    hnsw_pq.Insert(items[v])


In [4]:
from bruteforce import BruteForce

bf = BruteForce(items[:tot_items])
bf_res = bf.RunSearch(items[:tot_items], topk)

In [7]:
correct1 = 0
correct2 = 0

for i in range(tot_items):
    # print(hnsw_.KNNSearch(items[i], 10))
    res1 = hnsw_.KNNSearch(items[i], topk)
    res2 = hnsw_pq.KNNSearch(items[i], topk)
    for l, r in zip(res1, bf_res[i]):
        if l[0] == r[0] and l[1] == r[1]:
            correct1 += 1
    for l, r in zip(res2, bf_res[i]):
        if l[1] == r[1]: # PQ distance will always not be equivalent
            correct2 += 1
print("Recall@", topk, correct1 / (topk * tot_items))
print("RecallPQ@", topk, correct2 / (topk * tot_items))

# for l in range(hnsw_.graph.GetHeight()):
#     hnsw_.graph.PrintLayer(l)

Recall@ 10 1.0
RecallPQ@ 10 0.5015
